In [155]:
# Метод квадратов Шура-Адамара
def square(G):
    N = G.nrows()
    res = zero_matrix(F, N*(N-1)//2 + N, G.ncols())
    r = 0
    for i in range(N):
        for j in range(i, N):
            res[r] = G[i].pairwise_product(G[j])
            r += 1
    return res

In [156]:
# Матрица прозведений cтроки z на все строки матрицы G
def test(z,G):
    return matrix([row.pairwise_product(z) for row in G])

In [199]:
def make_key_pub(G,F,n,k,l):   
    import random
    gamma = random_matrix(F, 1, l)
    beta = random_matrix(F, 1, n)
    y = gamma.T*beta
    A = random_matrix(F, l, l)
    E = zero_matrix(F, l, n)
    d = vector(F, n) 
    for i in range(W):
        ind = random.randint(0, n - 1)
        d[ind] = 1
    for i in range(l):
        for j in range(n):
            if d[j] == 1:
                E[i, j] = F.random_element()
    print(E)
    A = random_matrix(F,l)
    E_pub = A*G[k-l:k]+E+y
    return E_pub, y, A

In [165]:
def encryption(E_pub, m, G):
    e = zero_matrix(F,1,n)
    for i in range(w):
        ind = random.randint(0,n-1)
        e[0,ind] = 1
    print(e)
    alpha = random_matrix(F,1,l)
    y = m*G + alpha*E_pub + e
    return y, alpha
    

In [166]:
def welsh_berlekamp(y_new,G_new):
    n1 = G_new.ncols()
    k1 = G_new.nrows()
    d = n1 - k1 + 1
    deg_N = k1 - 1 + (d-1) // 2
    deg_L = (d-1) // 2
    a_ = matrix(F, 1, n1)    # новый вектор a', в котором элементы совпадают со второй строчкой матрицы G'
    for i in range(n1):
        a_[0,i] = G_new[1,i]
    N = zero_matrix(F, n1, deg_N+1)                
    for i in range(n1):
        for j in range(deg_N+1):
            N[i,j]= a_[0,i]^j
    L=zero_matrix(F,n1,deg_L+1)
    for i in range(n1):
        for j in range(deg_L+1):
            L[i,j] = y_new[0,i]*a_[0,i]^j
    M = N.augment(-1*L)
    sol = M.right_kernel().random_element()
    N_coefs = sol[0:deg_N+1]
    L_coefs = sol[deg_N+1:]
    R.<x> = F[]
    N = sum([N_coefs[i] * x^i for i in range(deg_N+1)])
    L = sum([L_coefs[i] * x^i for i in range(deg_L+1)])
    if(L.is_zero()):
        return zero_matrix(F,1,k)
    U = N//L
    m_ = vector(F,U.list())
    return m_

In [167]:
def decode(y, G, E_pub, G_pub, alpha, y_rows):
    G_ = G_pub.stack(E_pub)
    ind_col = []
    rang_ = square(G_)
    for j in range (n):
        if(square(G_[:,  [i for i in range(n) if i != j]]).rank() != square(G_).rank()):
            ind_col.append(j)
    G_new = G[:, [i for i in range(n) if i not in ind_col]]
    G_pub_new = G_new[k-l:l]
    y_new = y[:, [i for i in range(n) if i not in ind_col]]
    y_rows = y_rows[:, [i for i in range(n) if i not in ind_col]]
    y_new = y_new - alpha*y_rows
    m_ = welsh_berlekamp(y_new, G_new)
    return(m_)

In [168]:
def attack(y,E_pub, G_pub, F, n, k, l):
    G_ = G_pub.stack(E_pub)
    ind_col = []
    rang_ = square(G_)
    for j in range (n):
        if(square(G_[:,  [i for i in range(n) if i != j]]).rank() != square(G_).rank()):
            ind_col.append(j)
    G1 = G_[:,  [i for i in range(n) if i not in ind_col]]
    G2 = G_pub[:,  [i for i in range(n) if i not in ind_col]]
    G3 = E_pub[:,  [i for i in range(n) if i not in ind_col]]
    z1 = random_vector(F, k-l)*G2
    z2 = random_vector(F, k-l)*G2
    lst = []
    for i in range(1000):
        z3 = random_vector(F, l)*G3
        p = test(z1,G1).stack(test(z2,G1).stack(test(z3,G1))).rank()
        if(p<=2*k-1+W):
            lst.append(z3)
    M = matrix(lst)
    G_new = G2.stack(M)
    for row in G3:
        G4 = G_new.stack(row)
        if G4.rank() == G_new.rank() + 1:
            break
    y_new = y[:,  [i for i in range(n) if i not in ind_col]]
    return G4, y_new, row, ind_col 

In [189]:
F = GF(31)
n = 31
k = 7
l = 3
W = 4
w = 5
import random
elements = list(F)
ksi = vector(F, random.sample(elements, n)) # Носитель RS-кода
G = zero_matrix(F,k,n)
for i in range(k):
    for j in range(n):
        G[i,j] = ksi[j]^i
G # Порождающая матрица

[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1]
[14  1 28  6 25  3 10 29  8  0  4 12  2 26 22  9 18 16 21 19 27 17 23 20  5 15 24 13  7 30 11]
[10  1  9  5  5  9  7  4  2  0 16 20  4 25 19 19 14  8  7 20 16 10  2 28 25  8 18 14 18  1 28]
[16  1  4 30  1 27  8 23 16  0  2 23  8 30 15 16  4  4 23  8 29 15 15  2  1 27 29 27  2 30 29]
[ 7  1 19 25 25 19 18 16  4  0  8 28 16  5 20 20 10  2 18 28  8  7  4  9  5  2 14 10 14  1  9]
[ 5  1  5 26  5 26 25 30  1  0  1 26  1  6  6 25 25  1  6  5 30 26 30 25 25 30 26  6  5 30  6]
[ 8  1 16  1  1 16  2  2  8  0  4  2  2  1  8  8 16 16  2  2  4  8  8  4  1 16  4 16  4  1  4]

In [190]:
E_pub, y_rows, A = make_key_pub(G,F,n,k,l)
E_pub # Публичный ключ

[16 22  0  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[22 16  5 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 7 17 26  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


[ 2 18 28 27 29  6 11  2 24  6 25  7  1 30  4 16  8  7 18 17 22 15 20  7 18  2 26 18 27  5 24]
[ 2 23  1  0 17 12  9 14 19 25 24 30 10 21  5 23 12 14  3 26  1  7 28  1  1 24 28  3  2  3 16]
[ 2  2  9 30  5 16 11 11  4  9 12 17 18  3  3  1  7 12 26 28 30 27  5 25 27 27 19 11  1 21 24]

In [191]:
m = random_matrix(F,1,k) # исходное сообщение 

m[0, 4:] = 0 # добавляем l нулей в конец сообщения
m

[13 26  3  0  0  0  0]

In [192]:
y,alpha = encryption(E_pub,m,G)
y,alpha # шифртекст и значения l элементов, которые были подставлены в конец сообщения m

[0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


([ 7 24  0 22 13 18 22  4 14 30 11 27 14 22  7  3 27 27  2 14 20 21 24 30 20 16 22 18 26  4  5],
 [ 1 13  3])

In [200]:
mes = decode(y, G, E_pub, G_pub, alpha, y_rows)
mes[0:4], m[:, :k-l] # Успешное декодирование легального пользователя

((13, 26, 3, 0), [13 26  3  0])

In [194]:
G_pub = G[:k-l]
G4, y_new, row, ind_col = attack(y,E_pub,G_pub,F,n,k,l)
G4, y_new, row, ind_col # найдены индексы (ind_col) зашумленых столбцов, а также зашумлёная строка (row) 

(1005 x 27 dense matrix over Finite Field of size 31,
 [13 18 22  4 14 30 11 27 14 22  7  3 27 27  2 14 20 21 24 30 20 16 22 18 26  4  5],
 (5, 16, 11, 11, 4, 9, 12, 17, 18, 3, 3, 1, 7, 12, 26, 28, 30, 27, 5, 25, 27, 27, 19, 11, 1, 21, 24),
 [0, 1, 2, 3])

In [195]:
G_1 = G[:,  [i for i in range(n) if i not in ind_col]]
G_1 # Порождающая матрица с удалёнными зашумлёнными столбцами

[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1]
[25  3 10 29  8  0  4 12  2 26 22  9 18 16 21 19 27 17 23 20  5 15 24 13  7 30 11]
[ 5  9  7  4  2  0 16 20  4 25 19 19 14  8  7 20 16 10  2 28 25  8 18 14 18  1 28]
[ 1 27  8 23 16  0  2 23  8 30 15 16  4  4 23  8 29 15 15  2  1 27 29 27  2 30 29]
[25 19 18 16  4  0  8 28 16  5 20 20 10  2 18 28  8  7  4  9  5  2 14 10 14  1  9]
[ 5 26 25 30  1  0  1 26  1  6  6 25 25  1  6  5 30 26 30 25 25 30 26  6  5 30  6]
[ 1 16  2  2  8  0  4  2  2  1  8  8 16 16  2  2  4  8  8  4  1 16  4 16  4  1  4]

In [196]:
for i in F:
    try:
        row_ = matrix(F,[row])
        m_ = welsh_berlekamp(y_new-i*row_,G_1)
        c = m_*G_1
        e_ = y_new - matrix(F,c) - i*row_
        if(e_[0].hamming_weight() <= w):
            break
    except:
        continue
e_,c # нашли маленькую ошибку, по которой определили подходящее кодовое слово c

([0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0],
 (7, 5, 14, 28, 3, 13, 9, 19, 11, 6, 22, 7, 30, 25, 8, 30, 15, 1, 18, 0, 0, 27, 24, 11, 0, 16, 1))

In [197]:
c_ = zero_matrix(F,1,len(c))
for i in range(len(c)):
    c_[0,i] = c[i]

In [201]:
y_new = y_new - e_
message = welsh_berlekamp(c_, G_1)
message[0:4],m[:, :k-l]  # Нелегальный пользователь верно восстановил исходное сообщение

((13, 26, 3, 0), [13 26  3  0])